In [144]:
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import os 
import pandas as pd
load_dotenv()

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
DB_PATH = '../backend/data.db'

conn = sqlite3.Connection(DB_PATH)
cur = conn.cursor()
# cur.execute("""
#     select chunk
#     from chunks
#     where article_id == ?
# """, (19017269,))
# chunks = ' '.join([x[0] for x in cur.fetchall()]).split()

pd.read_sql("""
    select *
    from articles 
    where count > 60
    order by random()
    limit 10
""", conn)

# pd.read_sql("select * from articles where title like '%seinfeld%' order by count desc""", conn)

# cur.fetchall()

,article_id,title,clean_title,url,count
0,408362,American Music Awards,american music awards,https://en.wikipedia.org/wiki/American%20Music...,146
1,4721,British Empire,british empire,https://en.wikipedia.org/wiki/British%20Empire,1920
2,64485,New Spain,new spain,https://en.wikipedia.org/wiki/New%20Spain,518
3,50556,James IV of Scotland,james iv of scotland,https://en.wikipedia.org/wiki/James%20IV%20of%...,148
4,72297,Lebensraum,lebensraum,https://en.wikipedia.org/wiki/Lebensraum,97
5,129195,Alnwick,alnwick,https://en.wikipedia.org/wiki/Alnwick,61
6,1181466,Treccani,treccani,https://en.wikipedia.org/wiki/Treccani,64
7,1007676,The Dean Martin Show,the dean martin show,https://en.wikipedia.org/wiki/The%20Dean%20Mar...,68
8,310824,Marshal of France,marshal of france,https://en.wikipedia.org/wiki/Marshal%20of%20F...,242
9,180466,EasyJet,easyjet,https://en.wikipedia.org/wiki/EasyJet,137


In [167]:
x = """	210560	Whey
2541904	Guanzhong
586015	Deloitte	
47700	Coral	
31329803 The Weeknd
463734	Public health
82061	Tina Turner	
191280	Private equity	
72890	Utah Jazz
52135	Pneumonia
4068	Blaise Pascal
62092	Trotskyism
	158548	Earth Day
100627	Emo
45772	Rocky
39626432	Edward Snowden
1776	Arthritis	
991	Absolute value
18717883	Jodie Foster
5261	Celtic music	
42898	Anthrax	
371248	Prussia	
163115	Interest rate
295855	Catechism
48146	Fossil fuel
36536119 ResearchGate
3969 Buckingham Palace	
23545010 Insane Clown Posse	
178690 Steamboat
980865 Rust Belt	
143912	Reindeer	
145804	Civilian
87219	LeVar Burton
265554	Igbo people
3254510	Scala (programming language)
189322	Jeb Bush
44940	World Heritage Site
19156272 Labor Day"""

ids = []
for line in x.split('\n'):
    article_id, title = line.strip().split(maxsplit=1)
    article_id = article_id.strip()
    ids.append(article_id)

import random 

while ids:
    id = random.choice(ids)
    ids.remove(id)
    print('{' + f'"article_id": {id}, "count": -1' + '}')

{"article_id": 39626432, "count": -1}
{"article_id": 463734, "count": -1}
{"article_id": 42898, "count": -1}
{"article_id": 4068, "count": -1}
{"article_id": 191280, "count": -1}
{"article_id": 163115, "count": -1}
{"article_id": 44940, "count": -1}
{"article_id": 87219, "count": -1}
{"article_id": 3254510, "count": -1}
{"article_id": 62092, "count": -1}
{"article_id": 371248, "count": -1}
{"article_id": 1776, "count": -1}
{"article_id": 210560, "count": -1}
{"article_id": 265554, "count": -1}
{"article_id": 23545010, "count": -1}
{"article_id": 52135, "count": -1}
{"article_id": 48146, "count": -1}
{"article_id": 158548, "count": -1}
{"article_id": 2541904, "count": -1}
{"article_id": 100627, "count": -1}
{"article_id": 295855, "count": -1}
{"article_id": 5261, "count": -1}
{"article_id": 3969, "count": -1}
{"article_id": 72890, "count": -1}
{"article_id": 47700, "count": -1}
{"article_id": 45772, "count": -1}
{"article_id": 145804, "count": -1}
{"article_id": 980865, "count": -1}
{"a

In [3]:
import sqlite3
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
import os 

load_dotenv()

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
DB_PATH = 'data/data.db'

def get_daily_word_vector_live(daily_word: int):
    conn = sqlite3.Connection(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        select chunk
        from chunks
        where article_id == ?
    """, (daily_word,))
    chunks = [x[0] for x in cur.fetchall()]
    result = client.embeddings.create(
        input=chunks,
        model="text-embedding-3-small"
    )
    return np.array([np.array(x.embedding) for x in result.data]).mean(axis=0)

In [5]:
get_daily_word_vector_live(71329952).shape

(1536,)

In [15]:
import sqlite3
import numpy as np

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()
# cur.execute("""
#     select a.title, c.chunk
#     from embeddings e
#     join articles a
#         using (article_id)
#     join chunks c
#         using (chunk_id)
#     where e.id == ?
# """, (167680,))
cur.execute("""
    select chunk_id, count(*)
    from chunks
    group by chunk_id
    order by count(*) desc
    limit 5
""")
for row in cur.fetchall():
    print(row)
conn.close()

(0, 2)
(1, 2)
(2, 2)
(3, 2)
(4, 2)


In [1]:
import sqlite3
import numpy as np

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()
cur.execute("""
    select *
    from articles
    where article_id == ?
""", (19017269,))
cur.fetchall()


[(19017269,
  'World population',
  'world population',
  'https://en.wikipedia.org/wiki/World%20population',
  67)]

In [122]:
conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

article_id = 44579187

cur.execute("""
    select e.vector, e.chunk_id, c.chunk
    from (
        select vector, chunk_id 
        from embeddings
        where article_id == ?
    ) as e
    join (
        select chunk_id, chunk
        from chunks
        where article_id == ?
    ) as c
        using (chunk_id)
""", (article_id, article_id))

guess = cur.fetchall()
guess_matrix = np.array([np.frombuffer(row[0]) for row in guess])

(159563,
 'both old and new anime titles both on home video and theatrically, aided by the Scotland Loves Anime film festival. From 2015 to 2018, Anime Limited served as the distributors for Funimation in the British Isles. History On 14 December 2012, Kazé UK representative and Scotland Loves Anime director Andrew Partridge launched the social media and website for Anime Limited, and stated that he would be launching a new anime distribution company in 2013, with',
 0.7591181392379761)

In [ ]:
numerator = guess_matrix @ target
denominator_rhs = np.sqrt(np.sum(target ** 2)) 
denominator_lhs = np.sqrt(np.sum(guess_matrix ** 2, axis=1))
denominator = denominator_lhs * denominator_rhs
distances = 1 - (numerator / denominator)
indices = np.argsort(distances)

[(guess[i][1], guess[i][2], distances[i]) for i in indices][: limit]

array([0.79301268, 0.75911814, 0.77018558, 0.78948745, 0.80729594,
       0.84301049, 0.84762888, 0.82576959, 0.82698408, 0.82828016,
       0.79397142, 0.77697281])

In [ ]:
import time

times = []

for _ in range(10000):
    start = time.time()
    numerator = guess @ target
    denominator_rhs = np.sqrt(np.sum(target ** 2)) 
    denominator_lhs = np.sqrt(np.sum(guess ** 2, axis=1))
    denominator = denominator_lhs * denominator_rhs
    distance = 1 - (numerator / denominator)
    times.append((time.time() - start) * 10000)
np.mean(times)

0.5156283378601074

In [101]:
from scipy.spatial.distance import cosine

times = []

for _ in range(10000):
    start = time.time()
    brute = []
    for i in range(12):
        brute.append(cosine(target, guess[i, :]))
    brute = np.array(brute)
    times.append((time.time() - start) * 10000)
np.mean(times)

2.6936564445495605

In [ ]:
import pandas as pd

first = True
for chunk in pd.read_csv('data/chunks_high_len.csv', chunksize=10000):
    if first:
        c = chunk.drop('chunk', axis=1)
        first = False
        continue
    c = pd.concat([chunk.drop('chunk', axis=1), c], axis=0)

In [6]:
import os
import pandas as pd 

a = pd.read_parquet('data/articles.parquet')

In [7]:
a

,article_id,url,title,tokens,lower_title
1042,35797415,https://en.wikipedia.org/wiki/South%20African%...,South African labour law,51609,south african labour law
39508,49770190,https://en.wikipedia.org/wiki/1917%20New%20Yea...,1917 New Year Honours,50575,1917 new year honours
3852,47807310,https://en.wikipedia.org/wiki/1943%20Birthday%...,1943 Birthday Honours,49553,1943 birthday honours
117787,47333075,https://en.wikipedia.org/wiki/Var%C3%A8se%20Sa...,Varèse Sarabande albums discography,46566,varèse sarabande albums discography
75220,7285706,https://en.wikipedia.org/wiki/Municipal%20hist...,Municipal history of Quebec,45717,municipal history of quebec
...,...,...,...,...,...
101402,147778,https://en.wikipedia.org/wiki/Music%20of%20Japan,Music of Japan,5001,music of japan
76948,1686010,https://en.wikipedia.org/wiki/Procession,Procession,5001,procession
49951,7031132,https://en.wikipedia.org/wiki/Perfume%3A%20The...,Perfume: The Story of a Murderer (film),5001,perfume: the story of a murderer (film)
6035,11494669,https://en.wikipedia.org/wiki/2005%20New%20Yor...,2005 New York Yankees season,5001,2005 new york yankees season


In [ ]:
# s = pd.DataFrame(chunks).sample().iloc[0]['chunk']
# start = 0
# end = 100
# while end < len(s):
#     print(s[start : end])
#     start = end
#     end += 100